# Import libraries

In [12]:
from azure.cosmos import CosmosClient, PartitionKey, exceptions
import json
import time

# Create client connection

In [13]:
endpoint = "https://smart-meter-benchmark.documents.azure.com:443/"
key = "HvrYXT5hnZmhRNJf2vJdXRy7Ms4YSKQE6lP77MAw4AES7N7sXLopbTyy1LoXYeBwRLgKEGms3S0JACDbKq9F3Q==;"
client = CosmosClient(endpoint, key)

# Create database and container

In [14]:
database_name = "smart-meter-benchmark_mm"

try:
    database = client.create_database(id=database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database=database_name)

In [15]:
container_id = "energy_mm"
partition_key_path = "/id_mm"

try:
    partition_key = PartitionKey(path=partition_key_path)
    container = database.create_container(
        id=container_id,
        partition_key=partition_key,
        offer_throughput=400,
    )
    print(f"Container created: {container.id}")

except exceptions.CosmosResourceExistsError:
    print("Container already exists.")

Container created: energy_mm


# Data Insertion

In [22]:
# Access the database and container
database_name = "smart-meter-benchmark_mm"
container_name = "energy_mm"
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

# Read json data
json_file = "/Users/jafarabdurrahmaan/Dropbox/00_Uni/Masterthesis/A_Thesis/1_Components/13_Benchmark_Study_Components/3_KVS/CosmosDB/iter2.2/daily_dataset_small_kv.json"

# Start measuring time
start_time = time.time()  

#Iterate over json file
with open(json_file, "r") as file:
    json_data = json.load(file)
    for item in json_data:
        item_id = item["id"]
        lclid = item["LCLid"]
        day = item["day"]
        energy_median = float(item["energy_median"])
        energy_mean = float(item["energy_mean"])
        energy_max = float(item["energy_max"])
        energy_count = int(item["energy_count"])
        energy_std = float(item["energy_std"])
        energy_sum = float(item["energy_sum"])
        energy_min = float(item["energy_min"])

        container_item = {
            "id": item_id,
            "LCLid": lclid,
            "day": day,
            "energy_median": energy_median,
            "energy_mean": energy_mean,
            "energy_max": energy_max,
            "energy_count": energy_count,
            "energy_std": energy_std,
            "energy_sum": energy_sum,
            "energy_min": energy_min,
            "partitionKey": "default"
        }

        container.create_item(container_item)

# Stop measuring time
end_time = time.time()

# Calculate execution time
execution_time = end_time - start_time
print(f"Data insertion took {execution_time:.2f} seconds")

Data insertion took 265.52 seconds
